In [72]:
import pandas as pd
import numpy as np
import astropy.utils.data
from astropy.coordinates import SkyCoord,concatenate
import astropy.units as u
import datetime as _datetime
from astropy.io import fits as _fits
from astropy.coordinates import FK5,ICRS
from astropy.io import ascii
from astropy.coordinates import solar_system_ephemeris
from astropy.time import Time

# Join results tables together

In [73]:
df_200219=pd.read_csv('200219_results.csv')
df_191213=pd.read_csv('191213g_results.csv')
final_results=pd.concat([df_200219,df_191213], axis=0)
final_results=final_results.reset_index(drop=True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [74]:
final_results_ps1=final_results[final_results['source']=='PS1']
final_results_glade=final_results[final_results['source']=='GLADE']

final_results_ps1[['our_id','RA','Dec']].to_csv('ps1_sources_for_gaia.csv')
final_results_glade[['our_id','RA','Dec']].to_csv('glade_sources_for_gaia.csv')
final_results[['our_id','source','RA','Dec']].to_csv('all_sources_for_gaia.csv')


# Load in xmatch

In [75]:
xmatch_ps1_only=pd.read_csv('gaia_ps1_xmatch_result.csv')
xmatch_all=pd.read_csv('all_xmatch_dr3.csv')

In [76]:
print(list(xmatch_all.columns))

['dist', 'solution_id', 'designation', 'source_id', 'random_index', 'ref_epoch', 'ra', 'ra_error', 'dec', 'dec_error', 'parallax', 'parallax_error', 'parallax_over_error', 'pm', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 'ra_dec_corr', 'ra_parallax_corr', 'ra_pmra_corr', 'ra_pmdec_corr', 'dec_parallax_corr', 'dec_pmra_corr', 'dec_pmdec_corr', 'parallax_pmra_corr', 'parallax_pmdec_corr', 'pmra_pmdec_corr', 'astrometric_n_obs_al', 'astrometric_n_obs_ac', 'astrometric_n_good_obs_al', 'astrometric_n_bad_obs_al', 'astrometric_gof_al', 'astrometric_chi2_al', 'astrometric_excess_noise', 'astrometric_excess_noise_sig', 'astrometric_params_solved', 'astrometric_primary_flag', 'nu_eff_used_in_astrometry', 'pseudocolour', 'pseudocolour_error', 'ra_pseudocolour_corr', 'dec_pseudocolour_corr', 'parallax_pseudocolour_corr', 'pmra_pseudocolour_corr', 'pmdec_pseudocolour_corr', 'astrometric_matched_transits', 'visibility_periods_used', 'astrometric_sigma5d_max', 'matched_transits', 'new_matched_tra

# Identify sources not crossmatched

In [77]:
common = final_results.merge(xmatch_all,on=['our_id','our_id'])

not_xmatched=final_results[(~final_results.our_id.isin(common.our_id))&(~final_results.our_id.isin(common.our_id))]


# Cut on astrometric error and proper motion

In [78]:
xmatch_all['uwe']=np.sqrt(xmatch_all['astrometric_chi2_al']/(xmatch_all['astrometric_n_good_obs_al']-5))
print(min(xmatch_all['uwe']), max(xmatch_all['uwe']), np.nanmin(xmatch_all['pmra']), np.nanmax(xmatch_all['pmdec']))
#and cut on pmra pmdec


0.8511615078820235 166.74739273060956 -75.2187536114139 69.89171461007075


In [79]:
print(np.mean(xmatch_all['uwe']),np.mean(xmatch_all['pmra']))

2.2004502193010658 -2.250466045871791


In [80]:
xmatch_ps1_only=xmatch_all[xmatch_all['source']=='PS1']
xmatch_GLADE_only=xmatch_all[xmatch_all['source']=='GLADE']

xmatch_cut=xmatch_ps1_only[(xmatch_ps1_only['uwe']>1.5) & (np.abs(xmatch_ps1_only['pmra']<3) | (xmatch_ps1_only['pmra']==np.nan)) & (np.abs(xmatch_ps1_only['pmdec']<3) | (xmatch_ps1_only['pmdec']==np.nan))]


# Recombine gaia-limited df with ps1/GLADE columns

In [81]:
xmatch_glade_ps1_cut=pd.concat([xmatch_cut,xmatch_GLADE_only], axis=0)
print(len(xmatch_glade_ps1_cut))

11067


# If there are duplicate PS1 crossmatches, only take closest one

In [82]:
def sep(ra,dec,ra1,dec1):
    #c1 = SkyCoord(ra=ra, '-69d45m22s', frame='icrs')
    c1=SkyCoord(ra*u.deg, dec*u.deg)
    c2=SkyCoord(ra1*u.deg, dec1*u.deg)
    sep = c1.separation(c2)
    return sep.arcsecond

xmatch_glade_ps1_cut['separation']=np.abs(sep(xmatch_glade_ps1_cut['ra'].values,xmatch_glade_ps1_cut['dec'].values,xmatch_glade_ps1_cut['ra.1'].values,xmatch_glade_ps1_cut['dec.1'].values))
xmatch_glade_ps1_cut=xmatch_glade_ps1_cut.sort_values('separation').drop_duplicates(subset='our_id', keep='first')

xmatch_glade_ps1_cut['RA']=xmatch_glade_ps1_cut['ra']
xmatch_glade_ps1_cut['Dec']=xmatch_glade_ps1_cut['dec']
#

# Merge xmatch with non-xmatch

In [83]:
total_cut_df = pd.concat([not_xmatched,xmatch_glade_ps1_cut])

In [84]:
print(list(total_cut_df.columns))


['Unnamed: 0', 'PGC', 'GWGC', 'HyperLEDA', '2MASS', 'SDSS', 'flag1', 'RA', 'Dec', 'dist', 'dist_err', 'z', 'Bmag', 'e_Bmag', 'AbsBMag', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'eKmag', 'flag2', 'flag3', 'source', 'objName', 'objNameHMS', 'objAltName1', 'objAltName2', 'objAltName3', 'objPopularName', 'objID', 'uniquePspsOBid', 'ippObjID', 'surveyID', 'htmID', 'zoneID', 'tessID', 'projectionID', 'skyCellID', 'randomID', 'batchID', 'dvoRegionID', 'processingVersion', 'objInfoFlag', 'qualityFlag', 'raStack', 'decStack', 'raStackErr', 'decStackErr', 'raMeanErr', 'decMeanErr', 'epochMean', 'posMeanChisq', 'cx', 'cy', 'cz', 'lambda', 'beta', 'l', 'b', 'nStackObjectRows', 'nStackDetections', 'nDetections', 'ng', 'nr', 'ni', 'nz', 'ny', 'Column1', 'Column2', 'gQfPerfect', 'gMeanPSFMag', 'gMeanPSFMagErr', 'gMeanPSFMagStd', 'gMeanPSFMagNpt', 'gMeanPSFMagMin', 'gMeanPSFMagMax', 'gMeanKronMag', 'gMeanKronMagErr', 'gMeanKronMagStd', 'gMeanKronMagNpt', 'gMeanApMag', 'gMeanApMagErr', 'gMeanApMagSt

# Template FITS

In [88]:
template_hdulist = _fits.open('WS2021A1-003_CatalogueTemplate.fits')
template_primary_hdu = template_hdulist[0]
template_hdu = template_hdulist[1]
template_column_names = [col.name for col in template_hdu.columns]
print(template_column_names)


['CNAME', 'TARGSRVY', 'TARGPROG', 'TARGCAT', 'TARGID', 'TARGNAME', 'TARGPRIO', 'TARGUSE', 'TARGCLASS', 'PROGTEMP', 'OBSTEMP', 'GAIA_ID', 'GAIA_DR', 'GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR', 'HEALPIX', 'IFU_SPAXEL', 'IFU_PA', 'IFU_DITHER', 'MAG_G', 'MAG_G_ERR', 'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR', 'GAIA_MAG_G', 'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'APS_WL_MIN', 'APS_WL_MAX', 'APS_Z', 'APS_SIGMA', 'APS_TEMPL_LIB', 'APS_TEMPL_LIB_NORM', 'APS_PPXF_WL_MIN', 'APS_PPXF_WL_MAX', 'APS_PPXF_MOM', 'APS_PPXF_DEG_ADD', 'APS_PPXF_DEG_MULT', 'APS_PPXF_NUM_MC', 'APS_GAND_MODE', 'APS_GAND_ERR', 'APS_GAND_RED1', 'APS_GAND_RED2', 'APS_GAND_EBV', 'APS_LS_MODE', 'APS_LS_RES', 'APS_LS_NUM_MC', 'APS_SSP_NUM_WLKR', 'APS_SSP_NUM_CHAIN', 'APS_IFU_MASK', 'APS_IFU_TSSL_TYPE', 'APS_IFU_TSSL_TARG_SNR', 'APS_IFU_TSSL_MIN_SNR', 'APS_IFU_TSSL_COVAR', 'APS_IFU_SRC_ID', 'APS_IF

In [89]:
from astropy.table import Table
Table.read(template_hdulist, hdu=1)

CNAME,TARGSRVY,TARGPROG,TARGCAT,TARGID,TARGNAME,TARGPRIO,TARGUSE,TARGCLASS,PROGTEMP,OBSTEMP,GAIA_ID,GAIA_DR,GAIA_RA,GAIA_DEC,GAIA_EPOCH,GAIA_PMRA,GAIA_PMRA_ERR,GAIA_PMDEC,GAIA_PMDEC_ERR,GAIA_PARAL,GAIA_PARAL_ERR,HEALPIX,IFU_SPAXEL,IFU_PA,IFU_DITHER,MAG_G,MAG_G_ERR,MAG_R,MAG_R_ERR,MAG_I,MAG_I_ERR,GAIA_MAG_G,GAIA_MAG_G_ERR,GAIA_MAG_BP,GAIA_MAG_BP_ERR,GAIA_MAG_RP,GAIA_MAG_RP_ERR,APS_WL_MIN,APS_WL_MAX,APS_Z,APS_SIGMA,APS_TEMPL_LIB,APS_TEMPL_LIB_NORM,APS_PPXF_WL_MIN,APS_PPXF_WL_MAX,APS_PPXF_MOM,APS_PPXF_DEG_ADD,APS_PPXF_DEG_MULT,APS_PPXF_NUM_MC,APS_GAND_MODE,APS_GAND_ERR,APS_GAND_RED1,APS_GAND_RED2,APS_GAND_EBV,APS_LS_MODE,APS_LS_RES,APS_LS_NUM_MC,APS_SSP_NUM_WLKR,APS_SSP_NUM_CHAIN,APS_IFU_MASK,APS_IFU_TSSL_TYPE,APS_IFU_TSSL_TARG_SNR,APS_IFU_TSSL_MIN_SNR,APS_IFU_TSSL_COVAR,APS_IFU_SRC_ID,APS_IFU_SRC_RA,APS_IFU_SRC_DEC,APS_FLAG,COORDCAT,COORDCAT_DR,COORDCAT_EPOCH,COORDCAT_ID,GENCAT_INFO,PS_DEC,PS_DR,PS_ID,PS_MAG_G,PS_MAG_G_ERR,PS_MAG_I,PS_MAG_I_ERR,PS_MAG_R,PS_MAG_R_ERR,PS_MAG_Y,PS_MAG_Y_ERR,PS_MAG_Z,PS_MAG_Z_ERR,PS_RA
,,,,,,,,,,,,,deg,deg,yr,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,,,deg,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,Angstrom,Angstrom,,km / s,,,Angstrom,Angstrom,,,,,,,,,,,Angstrom,,,,,,,,,,deg,deg,,,,yr,,,deg,,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,deg
str20,str15,str40,str30,str30,str30,float32,str1,str12,str8,str5,str25,str5,float64,float64,float32,float32,float32,float32,float32,float32,float32,int16,str6,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str10,int16,float32,float32,int16,int16,int16,int16,int16,int16,float32,float32,int16,int16,float32,int16,int16,int16,int16,str10,float32,float32,int16,str35,float64,float64,str48,str50,str10,float32,str30,str30,float64,str10,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64


## Extension 0 Header

In [90]:
ext0_header=template_hdulist[0].header

#Set the catalogue submitter and CCs

ext0_header['CAT_NME1']='Lana'
ext0_header['CAT_NME2']='Salmon'
ext0_header['CAT_MAIL']='lana.salmon@ucdconnect.ie'
ext0_header['CAT_CC']='p.jonker@sron.nl,morgan.fraser@ucd.ie,lorraine.hanlon@ucd.ie,antonio.martin-carrillo@ucd.ie'
print(repr(ext0_header))



SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT WS2021A1-003 Catalogue Template                                         
DATAMVER= '8.00    '           / WEAVE Data Model Version                       
TRIMESTE= '2021A1  '           / Observing Trimester                            
MAG_G_CM= '' / Survey specific mag column(s) used for MAG_G                     
MAG_R_CM= '' / Survey specific mag column(s) used for MAG_R                     
MAG_I_CM= '' / Survey specific mag column(s) used for MAG_I                     
STL_NME1= 'Peter   '                                                            
STL_NME2= 'Jonker  '                                                            
STL_MAIL= 'p.jonker@sron.nl'

## Extension 1 Header

In [91]:
ext1_header=template_hdulist[1].header
print(repr(ext1_header))

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  746 / length of dimension 1                          
NAXIS2  =                    0 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   88 / number of table fields                         
EXTNAME = 'WS2021A1-003 CATALOGUE' / extension name                             
TTYPE1  = 'CNAME   '           / WEAVE object name from coordinates             
TFORM1  = '20A     '           / data format of field: ASCII Character          
TDISP1  = 'A20     '           / Display format for column                      
TUCD1   = 'meta.id;meta.main

## Calculate gaia mag errors

In [92]:
# according to dr2 https://www.cosmos.esa.int/documents/29201/1773953/Gaia+DR2+primer+version+1.3.pdf/a4459741-6732-7a98-1406-a1bea243df79?t=1581668739161
total_cut_df['phot_g_mean_mag_err']=1.086/total_cut_df['phot_g_mean_flux_over_error']
total_cut_df['phot_bp_mean_mag_err']=1.086/total_cut_df['phot_bp_mean_flux_over_error']
total_cut_df['phot_rp_mean_mag_err']=1.086/total_cut_df['phot_rp_mean_flux_over_error']

## Take relevant columns from our catalogue

In [93]:
total_cut_df = total_cut_df.reset_index(drop=True)
FITS_table_from_cat=total_cut_df[['source','our_id','source_id','ra.1', 'dec.1', 'ref_epoch', 'pmra', 'pmra_error',
                                  'pmdec', 'pmdec_error','parallax', 'parallax_error','phot_g_mean_mag',
                                  'phot_g_mean_mag_err','phot_bp_mean_mag','phot_bp_mean_mag_err',
                                  'phot_rp_mean_mag','phot_rp_mean_mag_err', 'gMeanKronMag', 'gMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'objID', 'raMean', 'decMean',
                                  'gMeanKronMag', 'gMeanKronMagErr','iMeanKronMag', 'iMeanKronMagErr',
                                  'rMeanKronMag', 'rMeanKronMagErr','yMeanKronMag', 'yMeanKronMagErr','zMeanKronMag', 'zMeanKronMagErr']]


FITS_table_from_cat.columns = ['COORDCAT','TARGID','GAIA_ID','GAIA_RA', 'GAIA_DEC', 'GAIA_EPOCH', 'GAIA_PMRA', 'GAIA_PMRA_ERR', 
                               'GAIA_PMDEC', 'GAIA_PMDEC_ERR', 'GAIA_PARAL', 'GAIA_PARAL_ERR','GAIA_MAG_G', 
                               'GAIA_MAG_G_ERR', 'GAIA_MAG_BP', 'GAIA_MAG_BP_ERR', 
                               'GAIA_MAG_RP', 'GAIA_MAG_RP_ERR', 'MAG_G', 'MAG_G_ERR', 
                               'MAG_R', 'MAG_R_ERR', 'MAG_I', 'MAG_I_ERR',
                               'PS_ID', 'PS_RA', 'PS_DEC',
                               'PS_MAG_G', 'PS_MAG_G_ERR', 'PS_MAG_I', 'PS_MAG_I_ERR', 
                               'PS_MAG_R', 'PS_MAG_R_ERR', 'PS_MAG_Y', 'PS_MAG_Y_ERR', 'PS_MAG_Z', 'PS_MAG_Z_ERR' ]

FITS_table_from_cat.reset_index()
#survey identifier
FITS_table_from_cat['TARGSRVY']='WS2021A1-003'

#CNAME is set by WEAVE
FITS_table_from_cat['CNAME']=''

#optional personal name for programme/catalogue
FITS_table_from_cat['TARGPROG']='GW-galaxy'
FITS_table_from_cat['TARGCAT']='GW-galaxy'

#T for target, GALAXY
FITS_table_from_cat['TARGUSE']='T'
FITS_table_from_cat['TARGCLASS']='GALAXY'

# PROGTEMP for MOS LR 3x1020s 60min OB
FITS_table_from_cat['PROGTEMP']=11331

#OBSTEMP for max airmass=2.4, moon dist=90 deg, 
#max sky brightness=20.5, max seeing = 1.3'', transparency=0.8
FITS_table_from_cat['OBSTEMP']='GAFAD'

#TARGNAME is the same as unique id
FITS_table_from_cat['TARGNAME']=FITS_table_from_cat['TARGID']

#TARGPRIO = 10.0 for the moment
FITS_table_from_cat['TARGPRIO']=10.0

#Gaia DR
FITS_table_from_cat['GAIA_DR']='eDR3'
FITS_table_from_cat['PS_DR']='2'
#Optional coordinates provenance columns

#PS1 and GLADE V2
def version (df):
    if df['source'] == 'GLADE' :
        return 'V2'
    else:
        return 'DR2'
FITS_table_from_cat['COORDCAT_DR']=total_cut_df.apply (lambda total_cut_df: version(total_cut_df), axis=1)
#FITS_table_from_cat['COORDCAT_DR']='2'

#don't know epoch of GLADE, so just give epoch to PS1
FITS_table_from_cat['COORDCAT_EPOCH']=total_cut_df['epochMean']

#ID from GLADE or PS1
def cat_id(df):
    if df['source'] == 'GLADE' :
        return df['SDSS']
    else:
        return df['objID']
FITS_table_from_cat['COORDCAT_ID']=total_cut_df.apply (lambda total_cut_df: cat_id(total_cut_df), axis=1)


# Set APS / IFU columns to the default by setting them to null

FITS_table_from_cat['APS_PPXF_MOM']=''
FITS_table_from_cat['APS_FLAG'] =''
FITS_table_from_cat['IFU_SPAXEL']=''
FITS_table_from_cat['APS_Z']=''
FITS_table_from_cat['APS_TEMPL_LIB']=''
FITS_table_from_cat['APS_GAND_EBV']=''
FITS_table_from_cat['APS_GAND_ERR']=''
FITS_table_from_cat['APS_TEMPL_LIB_NORM']=''
FITS_table_from_cat['APS_IFU_TSSL_MIN_SNR'] =''
FITS_table_from_cat['IFU_PA'] =''
FITS_table_from_cat['IFU_DITHER'] =''
FITS_table_from_cat['APS_WL_MIN'] =''
FITS_table_from_cat['APS_IFU_TSSL_TYPE'] =''
FITS_table_from_cat['GENCAT_INFO'] =''
FITS_table_from_cat['APS_SSP_NUM_WLKR'] =''
FITS_table_from_cat['APS_LS_NUM_MC'] =''
FITS_table_from_cat['APS_PPXF_DEG_MULT'] =''
FITS_table_from_cat['APS_GAND_MODE'] =''
FITS_table_from_cat['APS_GAND_RED2'] =''
FITS_table_from_cat['APS_PPXF_NUM_MC'] =''
FITS_table_from_cat['APS_IFU_SRC_DEC']=''
FITS_table_from_cat['APS_IFU_SRC_ID'] =''
FITS_table_from_cat['APS_LS_RES'] =''
FITS_table_from_cat['APS_IFU_TSSL_COVAR'] =''
FITS_table_from_cat['APS_SSP_NUM_CHAIN'] =''
FITS_table_from_cat['HEALPIX'] =''
FITS_table_from_cat['APS_LS_MODE'] =''
FITS_table_from_cat['APS_WL_MAX'] =''
FITS_table_from_cat['APS_IFU_MASK'] =''
FITS_table_from_cat['APS_PPXF_DEG_ADD'] =''
FITS_table_from_cat['APS_PPXF_WL_MAX'] =''
FITS_table_from_cat['APS_IFU_TSSL_TARG_SNR'] =''
FITS_table_from_cat['APS_GAND_RED1'] =''
FITS_table_from_cat['APS_IFU_SRC_RA'] =''
FITS_table_from_cat['APS_SIGMA'] =''
FITS_table_from_cat['APS_PPXF_WL_MIN']=''

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

## If we don't have Gaia RA/Dec, transform PS1/GLADE to J2016 and put this into Gaia_RA, Gaia_dec columns

## Fill in source of Mag G, Mag R, Mag I columns

In [94]:
# give a list of the columns where the Mag_GRI column came from. 
mag_source_g=[]
mag_source_r=[]
mag_source_i=[]

for t in range(0,len(total_cut_df)):
    if total_cut_df['source'][t]=='GLADE':
        mag_source_g.append('')
        mag_source_r.append('')
        mag_source_i.append('')
    else:
        mag_source_g.append('PS_MAG_G')
        mag_source_r.append('PS_MAG_R')
        mag_source_i.append('PS_MAG_I')

ext0_header['MAG_G_CM']='|'.join(mag_source_g)
ext0_header['MAG_R_CM']='|'.join(mag_source_r)
ext0_header['MAG_I_CM']='|'.join(mag_source_i)

# Set TARGPRIO based on magnitude

# Study new dataframe

In [95]:
pd.set_option('display.max_columns', None)
FITS_table_from_cat=FITS_table_from_cat[template_column_names]
display(FITS_table_from_cat)

,CNAME,TARGSRVY,TARGPROG,TARGCAT,TARGID,TARGNAME,TARGPRIO,TARGUSE,TARGCLASS,PROGTEMP,OBSTEMP,GAIA_ID,GAIA_DR,GAIA_RA,GAIA_DEC,GAIA_EPOCH,GAIA_PMRA,GAIA_PMRA_ERR,GAIA_PMDEC,GAIA_PMDEC_ERR,GAIA_PARAL,GAIA_PARAL_ERR,HEALPIX,IFU_SPAXEL,IFU_PA,IFU_DITHER,MAG_G,MAG_G_ERR,MAG_R,MAG_R_ERR,MAG_I,MAG_I_ERR,GAIA_MAG_G,GAIA_MAG_G_ERR,GAIA_MAG_BP,GAIA_MAG_BP_ERR,GAIA_MAG_RP,GAIA_MAG_RP_ERR,APS_WL_MIN,APS_WL_MAX,APS_Z,APS_SIGMA,APS_TEMPL_LIB,APS_TEMPL_LIB_NORM,APS_PPXF_WL_MIN,APS_PPXF_WL_MAX,APS_PPXF_MOM,APS_PPXF_DEG_ADD,APS_PPXF_DEG_MULT,APS_PPXF_NUM_MC,APS_GAND_MODE,APS_GAND_ERR,APS_GAND_RED1,APS_GAND_RED2,APS_GAND_EBV,APS_LS_MODE,APS_LS_RES,APS_LS_NUM_MC,APS_SSP_NUM_WLKR,APS_SSP_NUM_CHAIN,APS_IFU_MASK,APS_IFU_TSSL_TYPE,APS_IFU_TSSL_TARG_SNR,APS_IFU_TSSL_MIN_SNR,APS_IFU_TSSL_COVAR,APS_IFU_SRC_ID,APS_IFU_SRC_RA,APS_IFU_SRC_DEC,APS_FLAG,COORDCAT,COORDCAT_DR,COORDCAT_EPOCH,COORDCAT_ID,GENCAT_INFO,PS_DEC,PS_DR,PS_ID,PS_MAG_G,PS_MAG_G_ERR,PS_MAG_I,PS_MAG_I_ERR,PS_MAG_R,PS_MAG_R_ERR,PS_MAG_Y,PS_MAG_Y_ERR,PS_MAG_Z,PS_MAG_Z_ERR,PS_RA
0,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_8,200129_8,10.0,T,GALAXY,11331,GAFAD,NaN,eDR3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_22,200129_22,10.0,T,GALAXY,11331,GAFAD,NaN,eDR3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_84,200129_84,10.0,T,GALAXY,11331,GAFAD,NaN,eDR3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_97,200129_97,10.0,T,GALAXY,11331,GAFAD,NaN,eDR3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_100,200129_100,10.0,T,GALAXY,11331,GAFAD,NaN,eDR3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27973,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_3967,191213_3967,10.0,T,GALAXY,11331,GAFAD,6.755197e+18,eDR3,297.107849,-25.981625,2016.0,-0.827047,0.120763,-6.148269,0.065600,0.110872,0.108706,,,,,NaN,NaN,NaN,NaN,NaN,NaN,17.512820,0.001141,18.011080,0.022690,16.858644,0.023358,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27974,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_946,200129_946,10.0,T,GALAXY,11331,GAFAD,2.686201e+18,eDR3,319.651000,-1.142200,2016.0,-0.980243,1.069365,0.744199,0.944376,1.418905,0.928253,,,,,NaN,NaN,NaN,NaN,NaN,NaN,19.745108,0.007729,18.548388,0.029424,17.096600,0.018318,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27975,,WS2021A1-003,GW-galaxy,GW-galaxy,191213_1617,191213_1617,10.0,T,GALAXY,11331,GAFAD,3.128824e+18,eDR3,104.980667,5.307053,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN,NaN,NaN,NaN,NaN,21.507786,0.034453,NaN,NaN,NaN,NaN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,GLADE,V2,NaN,NaN,,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27976,,WS2021A1-003,GW-galaxy,GW-galaxy,200129_3680,200129_3680,10.0,T,GALAXY,11331,GAFAD,2.689749e+18,eDR3,319.433000,0.